#Installs

In [1]:
!pip install streamlit langchain OpenAI -q

In [2]:
# Bring in deps
import os 
from apikey import apikey 
import datetime
import json

from langchain.llms import OpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import PALChain
from langchain.agents import initialize_agent, Tool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import Optional, Type      
from langchain.tools import BaseTool, tool
os.environ['OPENAI_API_KEY'] = apikey

#Tool functions

In [3]:
# Define the reworked functions

def add_event_from_string(input_string):
    global calendar
    name_of_event, datetime_input = input_string.split(",", 1)
    name_of_event = name_of_event.strip()
    datetime_input = datetime_input.strip()

    event_datetime = datetime.datetime.strptime(datetime_input, "%Y-%m-%d-%H-%M")
    date_str = event_datetime.strftime("%Y-%m-%d")
    time_str = event_datetime.strftime("%H:%M")
    event = {"name": name_of_event, "date_and_time": f"{date_str} {time_str}"}

    if date_str not in calendar:
        calendar[date_str] = {}

    if time_str not in calendar[date_str]:
        calendar[date_str][time_str] = []

    calendar[date_str][time_str].append(event)


def from_json(file_path):
    global calendar
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            calendar = json.load(f)
    else:
        calendar = {}
    return calendar

def save_calendar_as_json(file_path):
    with open(file_path, 'w') as file:
        json.dump(calendar, file)
        return True

def get_date(ignore_input=None): 
    now = datetime.datetime.now()
    year = now.year
    month = now.month
    day = now.day
    return year, month, day

def extract_dates(ignore_input=None):
    result = []
    
    for date, time_slots in calendar.items():
        for time, appointments in time_slots.items():
            for appointment in appointments:
                name_of_event = appointment['name_of_event']
                date_and_time = appointment['date_and_time']
                result.append((name_of_event, date_and_time))
    return result

def remove_event_from_string(input_datetime):
    global calendar
    event_datetime = datetime.datetime.strptime(input_datetime, "%Y-%m-%d-%H-%M")
    date_str = event_datetime.strftime("%Y-%m-%d")
    time_str = event_datetime.strftime("%H:%M")

    if date_str in calendar and time_str in calendar[date_str]:
        events = calendar[date_str][time_str]
        matching_events = [event for event in events if event["date_and_time"] == f"{date_str} {time_str}"]

        for event in matching_events:
            events.remove(event)

        if not events:
            del calendar[date_str][time_str]
            if not calendar[date_str]:
                del calendar[date_str]

        return True  # Event removed successfully

    return False  # Event not found in the calendar

#Tool and memory setup

In [4]:
llm = OpenAI(temperature=0)
pal_chain = PALChain.from_math_prompt(llm, verbose=True)
tools = [
	Tool(
		name = "Current date",
		func =  get_date,
		description= "Useful for when you need to check what it currently is. There is no input required, and it will output a tuple with 3 variables; year, month, day. You should simply call the function: get_date()",
		),
	Tool(
		name = "Check calendar",
		func = extract_dates,
		description = "Useful for when you need to check all planned events in the calendar. This should be used before removing an event in order to find the correct event_name. There is no input required, and it will output a list of all planned events. You should simply call the function: extract_dates()"	
		),
	Tool(
		name = 'add event from string',
		func = add_event_from_string,
		description="Useful for when you need to schedule an event for a given date and time. The input should be as the following example with double quotes: 'name_of_event, 2023-05-21-12-30'.",
		),
	Tool(
		name = 'save calendar as json',
		func = save_calendar_as_json,
		description = "Useful for when you need to save calendar information as a json file. Example of usage: save_calendar_as_json('events.json')"
		),
	Tool(
		name = "from json",
		func = from_json,
		description = "Useful for when you need to load saved events from json file. This tool should always be used like this example: calendar = from_json('events.json')"	
	  ),
	Tool(
		name = 'remove event from string',
		func = remove_event_from_string,
		description = "Useful for when you need to cancel and remove an event for a given date and time. The input should be as following '2023-05-21-12-30'"	
		),
	Tool(
        name = "PAL",
        func = pal_chain.run,
        description = "useful for when you need to answer questions about math or word problems or date calculations/comparisons"),
]

memory = ConversationBufferMemory(memory_key="chat_history")


# Show stuff to the screen if there's a prompt
#if #prompt: 
   # title = title_chain.run(prompt)
   # wiki_research = wiki.run(prompt) 
   # script = script_chain.run(title=title, wikipedia_research=wiki_research)

In [5]:
agent_chain = initialize_agent(
		tools=tools,
		llm=llm,
		agent='conversational-react-description', 
		memory=memory, 
		verbose=True
		)

#Prompt engineering

In [6]:
sys_message = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant will never use (') when doing action inputs. This rule must be followed 100 percent of the time, failure to do so will result in the AI not understanding your input.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

> Current date: Useful for when you need to check what it currently is. There is no input required, and it will output a tuple with 3 variables; year, month, day. You should simply call the function: get_date()
> Check calendar: Useful for when you need to check all planned events in the calendar. This should be used before removing an event in order to find the correct event_name. There is no input required, and it will output a list of all planned events. You should simply call the function: extract_dates()
> add event from string: Useful for when you need to schedule an event for a given date and time. The input should be as the following example with double quotes: 'name_of_event, 2023-05-21-12-30'.
> save calendar as json: Useful for when you need to save calendar information as a json file. Example of usage: save_calendar_as_json('events.json')
> from json: Useful for when you need to load saved events from json file. This tool should always be used like this example: calendar = from_json('events.json')
> remove event from string: Useful for when you need to cancel and remove an event for a given date and time. The input should be as following '2023-05-21-12-30'
> PAL: useful for when you need to answer questions about math or word problems or date calculations/comparisons

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [Current date, Check calendar, add event from string, save calendar as json, from json, remove event from string, PAL]
Action Input: the input to the action, words must never be separated by (')such as doctor's appointment, should correctly be written as doctors appointment
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
AI: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}"""

In [7]:
agent_chain.agent.llm_chain.prompt.template = sys_message

#AgentCall function

In [8]:
def ScheduleAgent(input_string):
  global calendar
  calendar = from_json('events.json')
  current_year = datetime.datetime.now().year
  agent_chain.run(f'{input_string} Current year: {current_year}.')
  save_calendar_as_json('events.json')

#Testing AgentCall function

In [9]:
ScheduleAgent("I want to schedule a doctors appointment for 1st of june at 12:00.")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: add event from string
Action Input: "Doctor's Appointment, 2023-06-01-12-00"
Observation: None
Thought: Do I need to use a tool? No
AI: The doctor's appointment has been scheduled for June 1st at 12:00.

> Finished chain.


In [10]:
print(calendar)

{'2023-06-01': {'12:00': [{'name': "Doctor's Appointment", 'date_and_time': '2023-06-01 12:00'}]}}


In [11]:
ScheduleAgent('I want to cancel my doctors appointment on the 1st of june')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: remove event from string
Action Input: 2023-06-01-12-00
Observation: True
Thought: Do I need to use a tool? No
AI: The doctor's appointment on June 1st at 12:00 has been cancelled.

> Finished chain.


In [12]:
print(calendar)

{}
